### advent callendar
##### Analysis

- KAM topicのクラスタリング
 テキストクラスタリング(e5)
 llm
- KAM topicの時系列分析
- KAM topicと業種
- 勘定科目からの予測
- スモールデータセットの提供(2021,2022,2023,2024の6月)業種を絞る リポジトリはベータとして提供
     - KAM項目の抽出
     - リスクテキスト項目の抽出
     - 会計数値の抽出

# TODO

jppfsのpreがない
bs_dfが0になる

# Get metadata

# setup

In [1]:
from time import sleep
from tqdm import tqdm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
DATA_PATH=Path("/Users/yusuke_noro/Documents/Projects/edinet_xbrl_prep/data")



In [2]:
from edinet_xbrl_prep.edinet_api import request_term, edinet_response_metadata, request_doc

# query edinet

In [3]:
res_results = request_term(api_key=your_api_key, start_date_str='2024-06-01', end_date_str='2024-06-05')

NameError: name 'your_api_key' is not defined

In [ ]:
edinet_response_metadata_obj=edinet_response_metadata()
edinet_response_metadata_obj.set_data(res_results)
filename = str(DATA_PATH/"data.jsonl")
edinet_response_metadata_obj.save(filename)

In [6]:

edinet_response_metadata_obj=edinet_response_metadata(
    filename = DATA_PATH/"data.jsonl",
    tse_sector_url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls",
    tmp_path = DATA_PATH
    )

yuho_df = edinet_response_metadata_obj.get_yuho_df()

In [7]:
#yuho_df.sector_label_33.value_counts()

In [8]:
yuho_df_filtered = yuho_df.query("sector_label_33 == '食料品'")

# Load

In [9]:
download_docid_list = yuho_df_filtered.docID.to_list()

In [6]:
res_results = []
(DATA_PATH / "xbrl_doc").mkdir(parents=True, exist_ok=True)
for docid in tqdm(download_docid_list):
    out_filename = DATA_PATH / "xbrl_doc" / (docid + ".zip")
    res_results.append(request_doc(api_key=your_api_key, docid=docid, out_filename_str=out_filename))
    sleep(0.5)
#out_filename = PROJPATH+"data/0_metadata/trial/res_results_doc_teisei_yuho_2407_v0830.pkl.cmp"
#joblib.dump(res_results_teisei_yuho, out_filename, compress=True) 

100%|██████████| 79/79 [03:34<00:00,  2.72s/it]


# Link

In [10]:
from edinet_xbrl_prep.link_base_file_analyzer import account_list_common
from edinet_xbrl_prep.fs_tbl import get_fs_tbl


In [25]:

account_list_common_obj = account_list_common(
    data_path=DATA_PATH,
    account_list_year="2024"
)
(DATA_PATH / "xbrl_doc_ext").mkdir(parents=True, exist_ok=True)



label:  844
label:  5073


In [12]:
import sys
arelle_temp_dir = Path(DATA_PATH/"tmp")
arelle_dir = arelle_temp_dir.joinpath('arelle_tmp')
arelle_dir.mkdir(exist_ok=True)
cmd_arelle_dir = '--xdgConfigHome=' + str(arelle_dir)
sys.argv.append(cmd_arelle_dir)

In [13]:
#from edinet_xbrl_prep.xbrl_parser_rapper import xbrl_parser_rapper

#### 財務数値の取り出し
有価証券報告書XBRLから財務数値を取得します。

In [14]:
fs_dict={
    'BS':["_BalanceSheet","_ConsolidatedBalanceSheet"],
    'PL':["_StatementOfIncome","_ConsolidatedStatementOfIncome"],
    'CF':["_StatementOfCashFlows","_ConsolidatedStatementOfCashFlows"],
    'SS':["_StatementOfChangesInEquity","_ConsolidatedStatementOfChangesInEquity"],
    'NOTES':["_Notes","_ConsolidatedNotes"],
    'report':["_CabinetOfficeOrdinanceOnDisclosure"]}

In [ ]:
from edinet_xbrl_prep.xbrl_parser_rapper import *
from edinet_xbrl_prep.link_base_file_analyzer import *
from edinet_xbrl_prep.utils import *
from edinet_xbrl_prep.fs_tbl import *

def get_fs_tbl2(account_list_common_obj,docid,zip_file_str,temp_path_str,role_keyward_list):
    linkbasefile_obj = linkbasefile(
        zip_file_str=zip_file_str,
        temp_path_str=temp_path_str
        )
    linkbasefile_obj.read_linkbase_file()
    linkbasefile_obj.check()
    linkbasefile_obj.make_account_label(account_list_common_obj,role_keyward_list)
    xbrl_data_df,log_dict = get_xbrl_rapper(
        docid=docid,
        zip_file=zip_file_str,
        temp_dir=temp_path_str,
        out_path=temp_path_str,
        update_flg=False
        )
    #role_keyward_list = []
    #print(list(linkbasefile_obj.account_tbl_role_dict.keys()))
    for role_t in role_keyward_list:
        role_list = [role_key for role_key in list(linkbasefile_obj.account_tbl_role_dict.keys()) if role_t in role_key]
    data_list = []
    for role in list(linkbasefile_obj.account_tbl_role_dict.keys()):
        #print(role)
        key_in_the_role:pd.Series = linkbasefile_obj.account_tbl_role_dict[role].key
        data=pd.merge(
            xbrl_data_df.query("key in @key_in_the_role"),
            linkbasefile_obj.account_tbl_role_dict[role],
            on='key',
            how='left')#.query("not key.str.contains('Abstract')")
        data = data.assign(docid=docid,role=role)
        
        data.assign(
            non_consolidated_flg=data.context_ref.str.contains('NonConsolidated'),
            current_flg=data.context_ref.str.contains('CurrentYear'),
            prior_flg=data.context_ref.str.contains('Prior1Year')
            
        )
        data_list.append(data)
    return pd.concat(data_list)

In [ ]:
import pandera as pa
from pandera.typing import DataFrame, Series

class fs_data(pa.DataFrameModel):
    #ParentChildLink
    """
    'key': taxonomy like 'jpcrp_cor:NetSales'
    'data_str': data (string) like '1000000'
    'decimals': 3桁の表示
    'precision': ???
    'context_ref' # T:-3, M:-6, B:-9
    'element_name'
    'unit' # JPY
    'period_type'
    'isTextBlock_flg'
    'abstract_flg'
    'period_start' # durationの場合 当期末日, instantの場合 None
    'period_end' # durationの場合 当期末日, instantの場合 当期末日
    'instant_date' # durationの場合 None, instantの場合 当期末日
    'end_date_pv' # durationの場合 前期末日, instantの場合 None
    'instant_date_pv' # durationの場合 None, instantの場合 前期対象日
    'scenario'
    'role'
    'label_jp'
    'order'
    'child_key'
    'docid']
    """
    key: Series[str]
    data_str: Series[str]
    decimals: Series[str]
    precision: Series[str]
    context_ref: Series[str]
    element_name: Series[str]
    unit: Series[str]
    period_type: Series[str] = pa.Field(isin=['instant','duration'],nullable=True) # 'instant','duration'
    isTextBlock_flg: Series[int] = pa.Field(isin=[0,1],nullable=True) # 0,1
    abstract_flg: Series[int] = pa.Field(isin=[0,1],nullable=True) # 0,1
    period_start: Series[str]
    period_end: Series[str]
    instant_date: Series[str]
    #end_date_pv: Series[str]
    #instant_date_pv: Series[str]
    scenario: Series[str]
    role: Series[str]
    label_jp: Series[str]
    order: Series[str]
    #child_key: Series[str]
    docid: Series[str]


In [38]:
fs_tbl_loader_obj_list = []
for docid in tqdm(download_docid_list[1:3]):
    fs_tbl_loader_obj = get_fs_tbl2(
        account_list_common_obj=account_list_common_obj,
        docid=docid,
        zip_file_str=DATA_PATH / "raw/xbrl_doc" / (docid + ".zip"),
        temp_path_str=DATA_PATH / "raw/xbrl_doc_ext" / docid,
        role_keyward_list=fs_dict['BS']+fs_dict['PL']+fs_dict['CF']+fs_dict['SS']+fs_dict['NOTES'],
    )
    fs_tbl_loader_obj_list.append(fs_tbl_loader_obj)


  0%|          | 0/2 [00:00<?, ?it/s]

org 20
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 6.11 s 


 50%|█████     | 1/2 [00:17<00:17, 17.74s/it]

['rol_BalanceSheet', 'rol_StatementOfIncome', 'rol_StatementOfCashFlows-indirect', 'rol_StatementOfChangesInEquity', 'rol_NotesStatementOfIncome', 'rol_NotesSegmentInformationEtcFinancialStatements-01', 'rol_NotesStatementOfCashFlows', 'rol_NotesBalanceSheet', 'rol_NotesStatementOfChangesInEquity', 'rol_NotesSegmentInformationEtcFinancialStatements-09', 'rol_NotesSegmentInformationEtcFinancialStatements-03']
rol_BalanceSheet
rol_StatementOfIncome
rol_StatementOfCashFlows-indirect
rol_StatementOfChangesInEquity
rol_NotesStatementOfIncome
rol_NotesSegmentInformationEtcFinancialStatements-01
rol_NotesStatementOfCashFlows
rol_NotesBalanceSheet
rol_NotesStatementOfChangesInEquity
rol_NotesSegmentInformationEtcFinancialStatements-09
rol_NotesSegmentInformationEtcFinancialStatements-03
org 37
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 14.69 s 


100%|██████████| 2/2 [00:47<00:00, 23.82s/it]

['rol_BalanceSheet', 'rol_ConsolidatedBalanceSheet', 'rol_StatementOfIncome', 'rol_ConsolidatedStatementOfIncome', 'rol_ConsolidatedStatementOfCashFlows-indirect', 'rol_StatementOfChangesInEquity', 'rol_ConsolidatedStatementOfChangesInEquity', 'rol_NotesConsolidatedStatementOfIncome', 'rol_NotesConsolidatedStatementOfComprehensiveIncome', 'rol_NotesStatementOfIncome', 'rol_NotesConsolidatedBalanceSheet', 'rol_NotesBalanceSheet', 'rol_NotesSignificantAccountingPoliciesForPreparationOfConsolidatedFinancialStatements', 'rol_NotesSegmentInformationEtcConsolidatedFinancialStatements-02', 'rol_NotesSegmentInformationEtcConsolidatedFinancialStatements-03', 'rol_NotesSegmentInformationEtcConsolidatedFinancialStatements-09', 'rol_NotesConsolidatedStatementOfChangesInEquity', 'rol_NotesSegmentInformationEtcConsolidatedFinancialStatements-04', 'rol_NotesSegmentInformationEtcConsolidatedFinancialStatements-01', 'rol_NotesConsolidatedStatementOfCashFlows']
rol_BalanceSheet
rol_ConsolidatedBalanceSh

In [45]:
pd.concat(fs_tbl_loader_obj_list)#.columns
#[['role','scenario']].value_counts()
#columns#.role.value_counts()
#.context_ref.value_counts()
#.role.value_counts()


,key,data_str,decimals,precision,context_ref,element_name,unit,period_type,isTextBlock_flg,abstract_flg,...,period_end,instant_date,end_date_pv,instant_date_pv,scenario,role,label_jp,order,child_key,docid
0,jppfs_cor:CashAndDeposits,2657108000,-3,None,Prior1YearInstant_NonConsolidatedMember,CashAndDeposits,JPY,instant,0,0,...,2023-04-01,2023-04-01,None,2023-03-31,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_BalanceSheet,現金及び預金,1.020101,jppfs_cor:CashAndDeposits,S100TNWB
1,jppfs_cor:CashAndDeposits,3873928000,-3,None,CurrentYearInstant_NonConsolidatedMember,CashAndDeposits,JPY,instant,0,0,...,2024-04-01,2024-04-01,None,2024-03-31,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_BalanceSheet,現金及び預金,1.020101,jppfs_cor:CashAndDeposits,S100TNWB
2,jppfs_cor:NotesReceivableTrade,31632000,-3,None,Prior1YearInstant_NonConsolidatedMember,NotesReceivableTrade,JPY,instant,0,0,...,2023-04-01,2023-04-01,None,2023-03-31,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_BalanceSheet,受取手形,1.020101,jppfs_cor:NotesReceivableTrade,S100TNWB
3,jppfs_cor:NotesReceivableTrade,46153000,-3,None,CurrentYearInstant_NonConsolidatedMember,NotesReceivableTrade,JPY,instant,0,0,...,2024-04-01,2024-04-01,None,2024-03-31,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_BalanceSheet,受取手形,1.020101,jppfs_cor:NotesReceivableTrade,S100TNWB
4,jppfs_cor:AccountsReceivableTrade,4658698000,-3,None,Prior1YearInstant_NonConsolidatedMember,AccountsReceivableTrade,JPY,instant,0,0,...,2023-04-01,2023-04-01,None,2023-03-31,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_BalanceSheet,売掛金,1.020101,jppfs_cor:AccountsReceivableTrade,S100TNWB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,jppfs_cor:ImpairmentLossEL,57000000,-6,None,CurrentYearDuration_NonConsolidatedMember,ImpairmentLossEL,JPY,duration,0,0,...,2024-04-01,None,2024-03-31,None,"[{""ja"":[""連結個別"",""非連結又は個別""],""en"":[""Consolidated ...",rol_NotesSegmentInformationEtcConsolidatedFina...,減損損失,1.010201,jppfs_cor:ImpairmentLossEL,S100TLP1
0,jpcrp_cor:DescriptionOfReportableSegmentsTextB...,"<p class=""smt_text6"" style=""orphans:0;widows:0...",None,None,CurrentYearDuration,DescriptionOfReportableSegmentsTextBlock,None,duration,1,0,...,2024-04-01,None,2024-03-31,None,None,rol_NotesSegmentInformationEtcConsolidatedFina...,NaN,1.020101,jpcrp_cor:DescriptionOfReportableSegmentsTextB...,S100TLP1
1,jpcrp_cor:DisclosureOfChangesInReportableSegme...,"<p class=""smt_text3"" style=""orphans:0;widows:0...",None,None,CurrentYearDuration,DisclosureOfChangesInReportableSegmentsTextBlock,None,duration,1,0,...,2024-04-01,None,2024-03-31,None,None,rol_NotesSegmentInformationEtcConsolidatedFina...,NaN,1.020101,jpcrp_cor:DisclosureOfChangesInReportableSegme...,S100TLP1
2,jpcrp_cor:ExplanationOfMeasurementsOfSalesProf...,"<p class=""smt_text6"" style=""orphans:0;widows:0...",None,None,CurrentYearDuration,ExplanationOfMeasurementsOfSalesProfitLossAsse...,None,duration,1,0,...,2024-04-01,None,2024-03-31,None,None,rol_NotesSegmentInformationEtcConsolidatedFina...,NaN,1.020102,jpcrp_cor:ExplanationOfMeasurementsOfSalesProf...,S100TLP1


In [50]:

#docid="S100TLB3"
fs_tbl_loader_obj_list = []
for docid in tqdm(download_docid_list[1:3]):
    fs_tbl_loader_obj = fs_tbl_loader(
        account_list_common_obj=account_list_common_obj,
        docid=docid,
        zip_file_str=DATA_PATH / "raw/xbrl_doc" / (docid + ".zip"),
        temp_path_str=DATA_PATH / "raw/xbrl_doc_ext" / docid,
        role_label_list=['BS', 'PL', 'report'],
    )
    fs_tbl_loader_obj_list.append(fs_tbl_loader_obj)



  0%|          | 0/2 [00:00<?, ?it/s]

[read_linkbase_file] done in 1.97 s 
org 20
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 9.57 s 


 50%|█████     | 1/2 [00:21<00:21, 21.24s/it]

[get_xbrl_rapper] done in 9.62 s 
[read_linkbase_file] done in 3.48 s 
org 37
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 13.28 s 


100%|██████████| 2/2 [00:48<00:00, 24.18s/it]

[get_xbrl_rapper] done in 10.28 s 


In [12]:
import pandas as pd

In [73]:
bs_df_list = []
for itr_fs_tbl_loader in fs_tbl_loader_obj_list:
    bs_df = itr_fs_tbl_loader.get_data(
        doc_name='report',
        term='all',
        Consolidated=True
        )
    bs_df_list.append(bs_df)
bs_df = pd.concat(bs_df_list).reset_index(drop=True)

169
197


In [74]:
bs_df.query("key.str.contains('BusinessRisks')")

,key,data_str,decimals,precision,context_ref,element_name,unit,period_type,isTextBlock_flg,abstract_flg,...,period_end,instant_date,end_date_pv,instant_date_pv,scenario,role,label_jp,order,child_key,docID
19,jpcrp_cor:BusinessRisksTextBlock,"<p style=""page-break-before:always; line-heigh...",None,None,FilingDateInstant,BusinessRisksTextBlock,None,instant,1,0,...,2024-06-20,2024-06-20,None,2024-06-19,None,rol_CabinetOfficeOrdinanceOnDisclosureOfCorpor...,事業等のリスク,1.020203,jpcrp_cor:BusinessRisksTextBlock,S100TNWB
96,jpcrp_cor:BusinessRisksTextBlock,"<p style=""page-break-before:always; line-heigh...",None,None,FilingDateInstant,BusinessRisksTextBlock,None,instant,1,0,...,2024-06-21,2024-06-21,None,2024-06-20,None,rol_CabinetOfficeOrdinanceOnDisclosureOfCorpor...,事業等のリスク,1.020203,jpcrp_cor:BusinessRisksTextBlock,S100TLP1


In [ ]:
def preproc(org_data):
    #org_data=org_data.copy()
    #org_data['data'] = org_data['data'].str.replace(',','')
    org_data['data'] = pd.to_numeric(org_data['data_str'], errors='coerce')
    org_data['data'] = org_data['data'].fillna(0)
    org_data['data'] = org_data.data.astype('int') # for case of data is string object (cannot read csv with dtype=int)
    org_data=org_data.assign(
        #key_tag=org_data.key_org.str.replace(':','_'),
        context_ref_len=(org_data.context_ref
                         .str.split('_',expand=True)
                         .notna().sum(axis=1) # count component separated by '_'
                         ),
        #data_credit=org_data.data
        )
    assert org_data.reset_index()[['key','context_ref']].duplicated().sum()==0
    #self.org_data=org_data
    return org_data

bs_df=preproc(bs_df)#.head(20)

In [19]:

bs_df2=bs_df.sort_values('context_ref_len',ascending=True).drop_duplicates(keep='first',subset='key')

#bs_df2=bs_df2.assign(data_credit=((bs_df2.balance=='credit')*2-1)*bs_df2.data)

#### テキストの取り出し

In [65]:
tmp=fs_tbl_loader_obj_list[1].xbrl_data_df

In [69]:
tmp.query("key.str.contains('BusinessRisks')")

,key,data_str,decimals,precision,context_ref,element_name,unit,period_type,isTextBlock_flg,abstract_flg,period_start,period_end,instant_date,end_date_pv,instant_date_pv,scenario
1719,jpcrp_cor:BusinessRisksTextBlock,"<p style=""page-break-before:always; line-heigh...",None,None,FilingDateInstant,BusinessRisksTextBlock,None,instant,1,0,None,2024-06-21,2024-06-21,None,2024-06-20,None


In [62]:
tmp=fs_tbl_loader_obj_list[1].linkbasefile_obj.parent_child_df.query("child_key.str.contains('jpcrp_cor:BusinessRisksTextBlock')")
#.role.value_counts()
#.query("role.str.contains('rol_Audits')")
#get_presentation_account_list_obj.export_label_to_taxonomi_dict()

In [64]:
tmp.iloc[0]#.role

parent_key                        jpcrp_cor:BusinessRisksHeading
child_key                       jpcrp_cor:BusinessRisksTextBlock
role           http://disclosure.edinet-fsa.go.jp/role/jpcrp/...
child_order                                                    1
Name: 22, dtype: object

In [16]:
pd.pivot_table(index='docID',columns='label',values='value',aggfunc='first')

TypeError: pivot_table() missing 1 required positional argument: 'data'

In [3]:
linkbasefile_obj=linkbasefile(
    zip_file_str=DATA_PATH / "xbrl_doc" / "S100TLB3.zip",
    temp_path_str=DATA_PATH / "xbrl_doc_ext"/"S100TLB3")
linkbasefile_obj.read_linkbase_file()
linkbasefile_obj.check()
linkbasefile_obj.make_account_label(account_list_common_obj)
#linkbasefile_obj.make_summary_tbl()
#linkbasefile_obj.make_account_label_common(account_list_common_obj) 


152
379
529


## analysis link

In [20]:
account_link_tracer_obj.search_keys(keyword_in_taxonomy='Sales')

['jpcrp_cor:NetSalesSummaryOfBusinessResults',
 'jppfs_cor:NetSalesAbstract',
 'jppfs_cor:NetSales',
 'jppfs_cor:CostOfSales',
 'jppfs_cor:CostOfSalesAbstract']

In [16]:
account_link_tracer_obj.get_child_keys_recursive(
    parent_key='jppfs_cor:NetSalesAbstract',
    role=role_text)

['jpcrp030000-asr_E03926-000:ArentIncome', 'jppfs_cor:NetSales']

# xbrl parse

In [46]:
docid="S100TLB3"
xbrl_data_df,log_dict=get_xbrl_rapper(
    docid=docid,
    zip_file=DATA_PATH / "xbrl_doc" / (docid + ".zip"),
    temp_dir=DATA_PATH / "tmp",
    out_path=DATA_PATH / "xbrl_doc_ext"/"S100TLB3",
    update_flg=False)


In [49]:
pl_key=linkbasefile_obj.account_tbl_role_dict['rol_BusinessResultsOfReportingCompany'].key

In [53]:
tmp=pd.merge(
    linkbasefile_obj.account_tbl_role_dict['rol_BusinessResultsOfReportingCompany'],
    xbrl_data_df.query("key in @pl_key"),
    on='key',
    how='outer')#.query("not key.str.contains('Abstract')")


In [ ]:
dict_add,log_dict=get_data(
    account_link_tracer_obj=account_link_tracer_obj,
    xbrl_obj=textblock_cur,
    include_tree_top_keyword=conf['include_tree_top_keyword'],
    role_text=role_text,
    include_keyword_list=conf['include_keyword_list'],
    exclude_keyword=conf['exclude_keyword'],
    keep_taxonomi_list=conf['keep_taxonomi_list'],
    docid=docid,
    edinet_code=edinet_code,
    period_end=period_end,
    period_start=period_start,
    tag=conf['tag'],
    log_dict=log_dict
    )